In [1]:
import pandas as pd
import numpy as np
import datetime
import sys

In [ ]:
data = pd.read_excel('./월별_시군구별_전력사용량.xlsx')
data

In [ ]:
# 업종이 산업용인 행 및 비교 목적의 합계 행만 추출

data_ind = data.loc[(data['계약종별'] == '산업용') | (data['계약종별'] == '합 계')].reset_index(drop=True)
data_ind

,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2004,서울특별시,강남구,산업용,17739000,16601000,16604000,15105000,14712000,15015000,14586000,15803000,14820000,15280000,15910000,17140000
1,2004,서울특별시,강남구,합 계,321528000,320093000,303005000,288127000,282270000,322233000,358516000,404959000,351642000,291019000,292913000,318108000
2,2004,서울특별시,강동구,산업용,11857000,11344000,11585000,11367000,11300000,11718000,12216000,12669000,11902000,11432000,11452000,11517000
3,2004,서울특별시,강동구,합 계,108161000,108410000,97979000,97105000,92328000,97944000,105343000,121981000,107799000,94144000,96995000,101065000
4,2004,서울특별시,강북구,산업용,3761000,3824000,3812000,3703000,3388000,3476000,3558000,3898000,3867000,3411000,3541000,3571000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10139,2025,제주특별자치도,제주시,합 계,322888448,320271800,291380530,285828242,0,0,0,0,0,0,0,0
10140,2025,제주특별자치도,서귀포시,산업용,15850618,15109147,14290658,16869724,0,0,0,0,0,0,0,0
10141,2025,제주특별자치도,서귀포시,합 계,224799779,242412995,211136738,212641161,0,0,0,0,0,0,0,0
10142,2025,세종특별자치시,세종시,산업용,177088740,170494967,182824605,176126078,0,0,0,0,0,0,0,0


In [ ]:
# 시도명 전처리
region_mapping = {
        '강원도': '강원',
        '경기도': '경기',
        '경상남도': '경남',
        '경상북도': '경북',
        '광주광역시': '광주',
        '대구광역시': '대구',
        '대전광역시': '대전',
        '부산광역시': '부산',
        '서울특별시': '서울',
        '울산광역시': '울산',
        '인천광역시': '인천',
        '전라남도': '전남',
        '전라북도': '전북',
        '제주특별자치도': '제주',
        '충청남도': '충남',
        '충청북도': '충북',
        '세종특별자치시': '세종'
    }

data_ind['시도'] = data_ind['시도'].map(region_mapping).fillna(data_ind['시도'])

idx = data_ind[data_ind['시도'] == '황해북도'].index
data_ind.drop(idx, inplace=True)

data_ind.reset_index(drop=True, inplace=True)
data_ind

In [ ]:
processed_data = []

for i in range(len(data_ind)):
    for month in range(1, 13):
        temp = {'month': datetime.date(data_ind.iloc[i]['연도'], month, 1).strftime('%Y-%m-%d'),
                'region': data_ind.iloc[i]['시도'], 
                'city': data_ind.iloc[i]['시군구'],
                'division': data_ind.iloc[i]['계약종별'],
                'power_kwh': data_ind.iloc[i][str(month) + '월']}
        
        processed_data.append(temp)

In [ ]:
processed_df = pd.DataFrame(processed_data).sort_values(by=['month', 'region']).reset_index(drop=True)
processed_df.to_excel('./month_city_power_kwh_data.xlsx', index=False)
processed_df

,month,region,city,division,power_kwh
0,2004-01-01,강원,강릉시,산업용,63675000
1,2004-01-01,강원,강릉시,합 계,138249000
2,2004-01-01,강원,고성군,산업용,3319000
3,2004-01-01,강원,고성군,합 계,21329000
4,2004-01-01,강원,동해시,산업용,133815000
...,...,...,...,...,...
121387,2025-12-01,충북,단양군,합 계,0
121388,2025-12-01,충북,청주시,산업용,0
121389,2025-12-01,충북,청주시,합 계,0
121390,2025-12-01,충북,증평군,산업용,0


In [ ]:
rate_df = pd.read_excel('./제조업_평균가동률.xlsx')
rate_df.head(12)

,산업별,제조업 평균가동률
0,2002.01,75.6
1,2002.02,77.1
2,2002.03,79.2
3,2002.04,79.6
4,2002.05,78.0
5,2002.06,76.4
6,2002.07,78.7
7,2002.08,79.4
8,2002.09,77.0
9,2002.10,77.4


In [ ]:
rate_index_df = pd.read_excel('./제조업_생산능력_및_가동률지수(2020=100).xlsx')
rate_index_df

,산업별,산업별 지수,제조업
0,2002.01,생산능력지수,58.654
1,2002.01,가동률지수(원지수),107.821
2,2002.01,가동률지수(계절조정),106.348
3,2002.02,생산능력지수,58.726
4,2002.02,가동률지수(원지수),94.685
...,...,...,...
835,2025.03,가동률지수(원지수),105.600
836,2025.03,가동률지수(계절조정),104.800
837,2025.04,생산능력지수,102.800
838,2025.04,가동률지수(원지수),105.600


In [ ]:
rate_index_df.loc[rate_index_df['산업별'] == rate_df.iloc[0]['산업별']]

,산업별,산업별 지수,제조업
0,2002.01,생산능력지수,58.654
1,2002.01,가동률지수(원지수),107.821
2,2002.01,가동률지수(계절조정),106.348


In [ ]:
processed_data = []

for i in range(len(rate_df)):
    temp = {'month': datetime.date(int(rate_df.iloc[i]['산업별']),  round((rate_df.iloc[i]['산업별'] - int(rate_df.iloc[i]['산업별']))*100), 1).strftime('%Y-%m-%d'),
            'util_rate': rate_df.iloc[i]['제조업 평균가동률'],
            'prod_cab_index': rate_index_df.loc[rate_index_df['산업별'] == rate_df.iloc[i]['산업별']].iloc[0]['제조업'],
            'util_rate_index(origin)': rate_index_df.loc[rate_index_df['산업별'] == rate_df.iloc[i]['산업별']].iloc[1]['제조업'],
            'util_rate_index(seasonal)': rate_index_df.loc[rate_index_df['산업별'] == rate_df.iloc[i]['산업별']].iloc[2]['제조업']
            }
        
    processed_data.append(temp)
processed_df = pd.DataFrame(processed_data)
processed_df

processed_df.to_excel('./year_month_util_rate.xlsx', index=False)